# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [130]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords', 'averaged_perceptron_tagger'])

from sqlalchemy import create_engine

import re
import numpy as np
import pandas as pd
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [131]:
# load data from database
engine = create_engine('sqlite:///disasterResponse.db')
df = pd.read_sql("SELECT * FROM disasterResponse", engine)

# related should be boolean, but it's not
df['related'].replace(2,1, inplace=True)

X = df.message.values
#remove id, genre, message, and original
Y = df[['related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']]

In [132]:
#validate change in related column
df.related.value_counts()

1    20094
0     6122
Name: related, dtype: int64

### 2. Write a tokenization function to process your text data

In [133]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        
        #reduce words to root form
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [134]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier())
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [135]:
# 1. Split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# 2. Train
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [136]:
Y_pred = pipeline.predict(X_test)


Y_pred

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [137]:
# run a classification report on each column
for index, col in enumerate(Y.columns):
    print(col)
    print(classification_report(Y_test[col], Y_pred[:,index]))

related
             precision    recall  f1-score   support

          0       0.59      0.43      0.50      1252
          1       0.84      0.91      0.87      3992

avg / total       0.78      0.79      0.78      5244

request
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      4328
          1       0.85      0.38      0.52       916

avg / total       0.88      0.88      0.86      5244

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5220
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5244

aid_related
             precision    recall  f1-score   support

          0       0.69      0.91      0.79      3085
          1       0.76      0.43      0.55      2159

avg / total       0.72      0.71      0.69      5244

medical_help
             precision    recall  f1-score   support

          0       0.93      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [138]:
# lookup current parameters
pipeline.get_params

<bound method Pipeline.get_params of Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])>

In [139]:
#lets add a couple of new hyperparameters
parameters = {
    'clf__n_estimators': [50, 100, 200],
    'clf__min_samples_split': [2, 3, 4]
}


cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] clf__min_samples_split=2, clf__n_estimators=50 ..................
[CV]  clf__min_samples_split=2, clf__n_estimators=50, score=0.25418395079387784, total= 1.1min
[CV] clf__min_samples_split=2, clf__n_estimators=50 ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.3min remaining:    0.0s


[CV]  clf__min_samples_split=2, clf__n_estimators=50, score=0.2676298097553998, total= 1.0min
[CV] clf__min_samples_split=2, clf__n_estimators=50 ..................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.4min remaining:    0.0s


[CV]  clf__min_samples_split=2, clf__n_estimators=50, score=0.25336194563662373, total=  59.2s
[CV] clf__min_samples_split=2, clf__n_estimators=100 .................


...

[CV]  clf__min_samples_split=2, clf__n_estimators=200, score=0.25236051502145923, total= 3.4min
[CV] clf__min_samples_split=3, clf__n_estimators=50 ..................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 20.7min remaining:    0.0s


[CV]  clf__min_samples_split=3, clf__n_estimators=50, score=0.2531826634244028, total=  51.4s
[CV] clf__min_samples_split=3, clf__n_estimators=50 ..................
[CV]  clf__min_samples_split=3, clf__n_estimators=50, score=0.26448290659419255, total=  52.1s
[CV] clf__min_samples_split=3, clf__n_estimators=50 ..................
[CV]  clf__min_samples_split=3, clf__n_estimators=50, score=0.25293276108726753, total=  52.5s
[CV] clf__min_samples_split=3, clf__n_estimators=100 .................
[CV]  clf__min_samples_split=3, clf__n_estimators=100, score=0.250464883421542, total= 1.6min
[CV] clf__min_samples_split=3, clf__n_estimators=100 .................
[CV]  clf__min_samples_split=3, clf__n_estimators=100, score=0.2723501644972107, total= 1.6min
[CV] clf__min_samples_split=3, clf__n_estimators=100 .................
[CV]  clf__min_samples_split=3, clf__n_estimators=100, score=0.25178826895565093, total= 1.5min
[CV] clf__min_samples_split=3, clf__n_estimators=200 .................
[CV] 

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 54.5min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__n_estimators': [50, 100, 200], 'clf__min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [140]:
Y_train.related.value_counts()

1    16102
0     4870
Name: related, dtype: int64

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [141]:
Y_pred = cv.predict(X_test)

In [142]:
# run a classification report on each column
for index, col in enumerate(Y.columns):
    print(col)
    print(classification_report(Y_test[col], Y_pred[:,index]))

related
             precision    recall  f1-score   support

          0       0.71      0.33      0.45      1252
          1       0.82      0.96      0.88      3992

avg / total       0.79      0.81      0.78      5244

request
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      4328
          1       0.90      0.43      0.58       916

avg / total       0.89      0.89      0.88      5244

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5220
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5244

aid_related
             precision    recall  f1-score   support

          0       0.73      0.93      0.82      3085
          1       0.83      0.50      0.63      2159

avg / total       0.77      0.75      0.74      5244

medical_help
             precision    recall  f1-score   support

          0       0.93      1.00      0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(cv, open('model.pkl','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.